In [54]:
from matplotlib import pyplot as plt
from scipy.optimize import curve_fit
import numpy as np
import pandas as pd
import plotly.express as px
import os
import sys
import plotly.graph_objects as go 
dir_path = os.path.abspath('')
sys.path.append(dir_path + '/../')
from labbiofisica import final_val

In [55]:
path = dir_path+'/data/'

spettriRAW = pd.DataFrame()
for i in ['0','1','2','3','4','5']:
    c = 'C' + i
    data = pd.read_csv(path + c + '.dat',header = None, skiprows=1,sep='\t')
    
    spettriRAW['λ'] = data.iloc[:, 0]
    spettriRAW[c] = data.iloc[:,1]

spettriRAW.tail()

,λ,C0,C1,C2,C3,C4,C5
55,205,-0.02454,-0.02172,-0.02031,-0.02233,-0.01931,-0.01872
56,204,-0.02265,-0.01981,-0.02289,-0.02221,-0.02085,-0.01986
57,203,-0.02256,-0.01913,-0.02492,-0.02548,-0.01868,-0.02221
58,202,-0.02118,-0.02115,-0.02197,-0.02395,-0.02055,-0.02537
59,201,-0.01932,-0.02146,-0.02278,-0.01982,-0.02226,-0.02139


In [56]:
N = 162 #numero residui BLG
l = 0.5 # lunghezza cammino ottico in cm
R = 8.314 # J/(mol * K)
T = 298.15 # K

In [57]:
# import concentrazioni* 1000000 #M

concentrazioni = np.array([0,1.529499455,3.093301471,4.668148484,6.130150005,7.735930045]) # M

concentrazioniBLG = np.array([4.05537E-06, 3.52052E-06, 3.58147E-06, 3.64439E-06, 3.66342E-06, 3.57871E-06]) # M

print('Concentrazione GuHCl [M]: ',concentrazioni)

Concentrazione GuHCl [M]:  [0.         1.52949946 3.09330147 4.66814848 6.13015    7.73593004]


In [58]:
import plotly.graph_objects as go

fig = go.Figure()
for conc,c in zip(['C0', 'C1', 'C2', 'C3', 'C4', 'C5'],concentrazioni):
    fig.add_trace(go.Scatter(
        x=spettriRAW['λ'],
        y=spettriRAW[conc],
        mode='lines',
        name=np.round(c,2)
    ))

fig.update_layout(
    xaxis_title='λ [nm]',
    yaxis_title='θ [deg]',
    # title='Dati raccolti',
    legend_title='C denaturante [M]',
    legend=dict(
        x=1,
        y=0,
        xanchor='right',
        yanchor='bottom',
        # font=dict(size=10),  # Make legend font smaller
    ),
    yaxis=dict(range=[-0.037, -0.013])
)

fig.write_image('./images/spettro_raccolto.png')
fig.write_html('./html/spettro_raccolto.html')
fig.show()

### APPAIO LE CODE

In [59]:
fig = go.Figure()

spettri = pd.DataFrame()

spettri['λ'] = spettriRAW['λ']

for i, conc in enumerate(['C0', 'C1', 'C2', 'C3', 'C4', 'C5']):
    λ = spettriRAW['λ']

    θ = spettriRAW[conc] * 100 / (concentrazioniBLG[i] * N * l)

    # APPAIO DELLE CODE
    mean = θ[λ > 250].mean()
    θ = θ - mean
    spettri[conc] = θ  # aggiorna la colonna

    fig.add_trace(go.Scatter(
        x=λ,
        y=θ,
        mode='lines',
        name=np.round(concentrazioni[i],2)
    ))

fig.update_layout(
    yaxis=dict(range=[-2000,100]),
    xaxis_title='λ',
    yaxis_title='θ [deg·cm² dmol⁻¹]',
    legend_title='C denaturante [M]',
    legend=dict(
        x=1,
        y=0,
        xanchor='right',
        yanchor='bottom'
    )
)
fig.show()

In [60]:
λ = spettri['λ']

λmin = 220
λmax = 227

imin,imax = λ[λ == λmin].index[0], λ[λ == λmax].index[0]
# print(imin,imax)

Λ = np.arange(λmax,λmin-1,-1)

# Seleziona solo le colonne delle concentrazioni (senza le colonne λ)
spettro_ristretto = spettri.loc[imax:imin, ['C0','C1', 'C2', 'C3', 'C4', 'C5']]
spettro_ristretto = spettro_ristretto.T
spettro_ristretto.columns = Λ
spettro_ristretto.insert(0, 'Concentrazione', spettro_ristretto.index)
spettro_ristretto.reset_index(drop=True, inplace=True)
spettro_ristretto['Concentrazione'] = concentrazioni
display(spettro_ristretto)


,Concentrazione,227,226,225,224,223,222,221,220
0,0.000000,-1102.029113,-1153.781861,-1223.800285,-1293.818710,-1394.279927,-1424.722720,-1610.423759,-1683.486462
1,1.529499,-1017.316045,-992.768605,-1038.356707,-1094.465141,-1147.066798,-1220.709118,-1360.980203,-1445.142854
2,3.093301,-879.354767,-913.825749,-965.532223,-1044.815483,-1044.815483,-1124.098743,-1279.218165,-1224.064593
3,4.668148,-741.542243,-711.053983,-795.743595,-799.131179,-836.394609,-809.293933,-843.169778,-822.844271
4,6.130150,-568.853863,-599.183749,-552.003926,-592.443774,-504.824103,-605.923723,-737.353229,-666.583495
5,7.735930,-577.144306,-639.239928,-497.799900,-552.996009,-477.101360,-621.991144,-515.048684,-628.890657


In [61]:
# fig = go.Figure()
colors = px.colors.sample_colorscale(px.colors.sequential.Plasma, len(Λ))

# for idx,i in enumerate(range(λmin,λmax)):
#     fig.add_trace(go.Scatter(
#         x=spettro_ristretto['Concentrazione'],
#         y=spettro_ristretto[i],
#         mode='lines+markers',
#         name=str(i)+'nm',
#         line=dict(color=colors[idx]),
#         marker=dict(color=colors[idx])
#     ))

# fig.update_layout(
#     # yaxis_type='log',
#     #title='Spettri ristretti (220-230 nm)',
#     xaxis_title='Concentrazione [M]',
#     yaxis_title='θ',
#     legend_title='λ [nm]',
#     height=800,
#     width=600,
#     legend=dict(
#         x=1,
#         y=0,
#         xanchor='right',
#         yanchor='bottom'
#     )
# )
# # fig.show()

In [62]:
def sigmoid(x,m,Cmid, yN,yD):
    R = 8.314  # J/(mol * K)
    T = 300  # K
    expo = m * (Cmid-x) / (R*T)
    num = yN + yD * np.exp(-expo)
    den = 1 + np.exp(-expo)
    return num/den


# x = np.linspace(0,6,1000)
# plt.plot(x,sigmoid(x,-0.005,-0.0001,2,6))

# plt.show()

In [63]:
# fit con sigmoide

fit_results = []

for i in spettro_ristretto.columns[1:]:
    C = concentrazioni
    θ = spettro_ristretto[i].to_numpy()

    # guess
    yD = θ[-1]
    yN = θ[0] # NON è UN PARAMETRO
    m = 2666
    Cmid = np.mean(C)

    # fit
    print(i)
    popt, pcov = curve_fit(sigmoid, C, θ, p0=[m, Cmid, yN, yD],maxfev=100000,sigma=25)
    perr = np.sqrt(np.diag(pcov))
    fit_results.append({
        'λ': i,
        'm': popt[0], 'm_err': perr[0],
        'Cmid': popt[1], 'Cmid_err': perr[1],
        'Cm covariance': pcov[1, 1],
        'yN': popt[2], 'yN_err': perr[2],
        'yD': popt[3], 'yD_err': perr[3],
    })

fit_df = pd.DataFrame(fit_results)
display(fit_df)

227
226
225
224
223
222
221
220


,λ,m,m_err,Cmid,Cmid_err,Cm covariance,yN,yN_err,yD,yD_err
0,227,1852.922807,748.047370,3.546255,0.479887,0.230291,-1136.272827,78.762399,-531.365154,63.054650
1,226,1840.821985,1222.769991,2.886944,0.898735,0.807725,-1199.934654,167.400637,-590.009050,82.856803
2,225,1085.182543,1079.351890,4.114205,1.337248,1.788233,-1350.519860,369.781602,-296.196178,432.663479
3,224,1483.173787,1054.770389,3.858965,0.887477,0.787615,-1342.982450,208.868803,-449.993434,206.765546
4,223,1142.695742,1208.568273,3.798712,1.384868,1.917859,-1556.200717,525.745224,-258.928047,498.418828
5,222,2050.141305,1071.858298,3.555508,0.592639,0.351221,-1430.704565,136.951268,-560.419810,111.814806
6,221,1142.294819,1080.685470,3.874345,1.234064,1.522914,-1808.228021,531.653245,-303.594345,530.548438
7,220,1977.923403,601.172642,3.137134,0.368412,0.135728,-1750.480425,127.052278,-580.755845,76.812745


In [64]:
fig = go.Figure()

C_plot = np.linspace(concentrazioni.min(), concentrazioni.max(), 200)

for idx, row in fit_df.iterrows():
    yN = row['yN']#spettro_ristretto.loc[0, row['λ']]
    yD = row['yD'] #spettro_ristretto.loc[5, row['λ']]
    # print(yN, yD)

    m = row['m']
    Cmid = row['Cmid']
    θ_fit = sigmoid(C_plot, m, Cmid, yN, yD)
    # Fit line
    fig.add_trace(go.Scatter(
        x=C_plot,
        y=θ_fit,
        mode='lines',
        line=dict(color=colors[idx]),
        name=f"Fit {int(row['λ'])} nm"
    ))
    # Experimental points
    fig.add_trace(go.Scatter(
        x=spettro_ristretto['Concentrazione'],
        y=spettro_ristretto[row['λ']],
        mode='markers',
        marker=dict(color=colors[idx]),
        name=f"Exp {int(row['λ'])} nm",
        showlegend=False
    ))

fig.update_layout(
    xaxis_title='Concentrazione [μM]',
    yaxis_title='θ (fit)',
    legend_title='λ [nm]',
    height=600,
    width=800,
    legend=dict(
        x=1,
        y=0,
        xanchor='right',
        yanchor='bottom'
    )
)

fig.show()

In [65]:
# Extract Cmid and its error
Cmid_values = fit_df['Cmid']
Cmid_errors = fit_df['Cmid_err']

# Compute mean and standard error of the mean
weights = 1/Cmid_errors**2
mean_Cmid = np.average(Cmid_values, weights=weights)
mean_Cmid_err = np.sqrt(np.sum(weights*((Cmid_values-mean_Cmid)**2))/np.sum(weights))



fig = go.Figure()

# markers
fig.add_trace(go.Scatter(
    x=fit_df['λ'],
    y=Cmid_values,
    error_y=dict(type='data', array=Cmid_errors, visible=True),
    mode='markers',
    name='Cmid'
))

mean_line_points = np.concatenate([[219],fit_df['λ'],[231]])
# mean dashed line
fig.add_trace(go.Scatter(
    x= mean_line_points,
    y=[mean_Cmid]*len(mean_line_points),
    mode='lines',
    line=dict(color='red', dash='dash'),
    name='Mean Cmid'
))

# bottom
fig.add_trace(go.Scatter(
    x=mean_line_points,
    y=[mean_Cmid - mean_Cmid_err]*len(mean_line_points),
    mode='lines',
    line=dict(color='red', width=0),
    showlegend=False
))
#top
fig.add_trace(go.Scatter(
    x=mean_line_points,
    y=[mean_Cmid + mean_Cmid_err]*len(mean_line_points),
    mode='lines',
    fill='tonexty',
    fillcolor='rgba(255,0,0,0.2)',
    line=dict(color='red', width=0),
    name='Mean ± SEM'
))

fig.update_layout(
    xaxis_title='λ [nm]',
    yaxis_title='Cmid',
    # title='Cmid vs λ with Mean and SEM',
    legend_title='Legend',
    yaxis=dict(range=[0, 2]),
    legend=dict(
        x=1,
        y=1,
        xanchor='right',
        yanchor='top'
    )
)

fig.show()

In [66]:
# Extract m and its error
m_values = fit_df['m']
m_errors = fit_df['m_err']

# Compute mean and standard error of the mean
weights_m = 1 / m_errors**2
mean_m = np.average(m_values, weights=weights_m)
mean_m_err = np.sqrt(np.sum(weights_m * ((m_values - mean_m) ** 2)) / np.sum(weights_m))

fig = go.Figure()

# markers with error bars
fig.add_trace(go.Scatter(
    x=fit_df['λ'],
    y=m_values,
    error_y=dict(type='data', array=m_errors, visible=True),
    mode='markers',
    name='m'
))

mean_line_points = np.concatenate([[λmin-1], fit_df['λ'], [λmax+1]])
# mean dashed line
fig.add_trace(go.Scatter(
    x=mean_line_points,
    y=[mean_m]*len(mean_line_points),
    mode='lines',
    line=dict(color='red', dash='dash'),
    name='Mean m'
))

# bottom
fig.add_trace(go.Scatter(
    x=mean_line_points,
    y=[mean_m - mean_m_err]*len(mean_line_points),
    mode='lines',
    line=dict(color='red', width=0),
    showlegend=False
))
# top
fig.add_trace(go.Scatter(
    x=mean_line_points,
    y=[mean_m + mean_m_err]*len(mean_line_points),
    mode='lines',
    fill='tonexty',
    fillcolor='rgba(255,0,0,0.2)',
    line=dict(color='red', width=0),
    name='Mean ± SEM'
))

fig.update_layout(
    xaxis_title='λ [nm]',
    yaxis_title='m',
    legend_title='Legend',
    yaxis=dict(range=[0, 5000]),
    legend=dict(
        x=1,
        y=1,
        xanchor='right',
        yanchor='top'
    )
)

fig.show()


In [67]:
# stampa medie

# print("Media pesata di Cmid:", mean_Cmid)
# print("Errore sulla media pesata di Cmid:", mean_Cmid_err)
# print("Media pesata di m:", mean_A)
# print("Errore sulla media pesata di m:", mean_A_err)

print('Cmid: ',final_val(mean_Cmid,mean_Cmid_err,4,udm='M'))
print('m: ', final_val(mean_m,mean_m_err, udm='LJ/mol2'))

ΔG = m * mean_Cmid
sigmaΔG = ΔG*np.sqrt((mean_m_err/mean_m)**2 + (mean_Cmid_err/mean_Cmid)**2)

print(ΔG,sigmaΔG)

Cmid:  3.4004 ± 0.3077 M
m:  1685.04 ± 360.29 LJ/mol2
6725.721093129976 1561.5414905662954
